# Using a SurrealDB Knowedge Graph and Together AI hosted DeepSeek model to build a Graph-RAG 

This Jupyter Notebook illustrates the implementation and utilization of a graph-based Retrieval-Augmented Generation (RAG) system using a knowledge graph stored in SurrealDB. It focuses on leveraging the knowledge graph created by the graph_extractor.ipynb notebook for question-answering tasks.

### Key Functionality
####  Data Loading:
        Loads the raw text data from which will be used for context in the prompt.

#### Leveraging the Existing Knowledge Graph:

        Create a SurrQL function to query the knowledge graph using vector search for the descriptions of the entities and the other entities and relationships they have in the graph. This data will also be fed into the prompt for context.

        
#### Response Generation without a knowledge graph:
    Using a template prompt in the RAG_prompts.py file a simple context window of the text data loaded earlier with the question is asked of the LLM model.

    

#### Response Generation with knowledge graph:
    Using a template prompt in the RAG_prompts.py file a context window of the text data loaded earlier, plus the graph data retrieved from SurrealDB with the question is asked of the LLM model.


##### notes:
    This notebook utilizes libraries :
        surrealdb to interact with SurrealDB
        Together AI https://www.together.ai/

    Prerquisites:
            load an embedding model into Surreal: https://github.com/apireno/surrealDB_embedding_model
            run the chunking and embedding script as the context window is too small for the entire doc: chunking_and_embedding.ipynb
            load a knowledge graph into Surreal: (graph_extractor.ipynb in this Repo)
            create an account at together AI: https://api.together.ai/
            the setup at together will instruct you to store your API key in an env. variable.
            install the together python client: pip install together
            
            

    The notebook is loosely based on a larger repository microsoft/graphrag. The microsoft/graphrag repository provides a more comprehensive, modular graph-based RAG system. This notebook focuses specifically on the knowledge graph construction aspect.

In [1]:
import sys
import os
import json
from together import Together
import os
import time
import ipynb_path
import pandas as pd
import re
from surrealdb import AsyncSurrealDB

from IPython.display import display, Markdown,JSON
from IPython.display import JSON


#get this notebook's path for access to the other files needed
dir_path = os.path.dirname(os.path.realpath(ipynb_path.get(__name__)))
sys.path.append(dir_path) #add the current directory for adding py imports

#This python file holds the prompt templates
from RAG_prompts import DEEPSEEK_TOGETHERAI_STD_RAG_PROMPT, DEEPSEEK_TOGETHERAI_GRAPH_RAG_PROMPT

In [79]:

# this folder
nb_folder = dir_path
input_file = nb_folder + "/Operation Dulce v2 1 1.md"

out_folder = nb_folder + "/deepseek_togetherai_rag_{0}".format(time.strftime("%Y%m%d-%H%M%S"))
os.makedirs(out_folder, exist_ok=True)
chat_file = out_folder + "/chat.json"


#this variable is used to log debugging info
messages_to_log = []


#the SurrealDB hosting information
ip = "0.0.0.0:8000"
url = "ws://{0}".format(ip)

u = "root"
p = "root"
n = "graph_rag"
d = "graph_rag"
db_folder = nb_folder + "/db"

surrealdb_start = "surreal start --allow-net --log none --user {u} --pass {p} --bind {ip} \"rocksdb://{db_folder}\"".format(
    u=u,
    p=p,
    ip=ip,
    db_folder=db_folder)

print("""
run this command if your surreal instance isn't running yet 
copy and paste from below into a terminal
""")

print(surrealdb_start)

#and ensure you installed the embedding model!
#the model will power the function fn::sentence_to_vector($text)
print("""
ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)

make sure to chunk the document! chunking_and_embedding.pynb
    
"""   )  





run this command if your surreal instance isn't running yet 
copy and paste from below into a terminal

surreal start --allow-net --log none --user root --pass root --bind 0.0.0.0:8000 "rocksdb:///Users/sandro/git_repos/graph_rag/db"

ensure you installed the embedding model!
      
      https://github.com/apireno/surrealDB_embedding_model

the model will power the function fn::sentence_to_vector($text)

make sure to chunk the document! chunking_and_embedding.pynb
    



In [38]:

# get the text from the source file for use in our prompt
with open(input_file, "r") as f:
    input_text = f.read()


entity_types = ["PERSON", "PLACE"]
relation_type = "RELATED_TO"

#create the function that will return the portion of the knowledge graph 
#relevant to the question using vector search on the description embedding
#we will use python format to replace the table names in our funtion with the entity types and relation type strings
retrieve_function_surql = """
DEFINE FUNCTION OVERWRITE fn::get_related_entities($text: string) {{
    
    LET $emb = fn::sentence_to_vector($text);
    
    RETURN SELECT id,description, ->{1}.{{out,strength,description}} FROM {0} 
        WHERE  embedding <|5,COSINE|> $emb;
}};
""".format(entity_types[0],relation_type)

TABLE_NAME = "CHUNKS"

retrieve_chunks_surql = """
DEFINE FUNCTION OVERWRITE fn::get_related_chunks($text: string) {{
    LET $emb = fn::sentence_to_vector($text);
    RETURN SELECT chunk FROM {TABLE_NAME} WHERE  embedding <|5,COSINE|> $emb;
}};
""".format(TABLE_NAME = TABLE_NAME)



#execute the DDL of the function
async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    outcome = await db.query(retrieve_function_surql)
    outcome = await db.query(retrieve_chunks_surql)



In [4]:

#This is a helper function to make the knowledge graph a bit easier to read.
def transform_record_ids(data):
  """
  Transforms the input data to extract the 'ID' field from 'out' and 'in' objects.

  Args:
    data: An array of dictionaries.

  Returns:
    The transformed array of dictionaries.
  """
    
  for item in data:
    if "id" in item:
        item["id"] = item["id"].id
    if "->RELATED_TO" in item:
      for inner_dict in item["->RELATED_TO"]:
        if "out" in inner_dict and hasattr(inner_dict["out"], "id"):
          inner_dict["out"] = inner_dict["out"].id
        if "in" in inner_dict and hasattr(inner_dict["in"], "id"):
          inner_dict["in"] = inner_dict["in"].id
        if "id" in inner_dict and hasattr(inner_dict["id"], "id"):
          inner_dict["id"] = inner_dict["id"].id
      
    
  return data


In [5]:

#Let's test out vector/graph query
#we should see a collection of people and relationships
#where the person's description is related to the question 

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    outcome = await db.query(get_the_graph_context_surql.format("Who is the main character?"))

result = transform_record_ids(outcome[0]["result"])


print(json.dumps(result, indent=4, sort_keys=True))
    

[
    {
        "->RELATED_TO": [
            {
                "description": "Alex Mercer operates within Dulce Base during Operation: Dulce.",
                "out": "DULCE BASE",
                "strength": 9
            },
            {
                "description": "Alex Mercer and Taylor Cruz are both members of the Paranormal Military Squad and work together on Operation: Dulce.",
                "out": "TAYLOR CRUZ",
                "strength": 8
            },
            {
                "description": "Alex Mercer reports to and receives orders from Washington D.C.",
                "out": "WASHINGTON",
                "strength": 6
            },
            {
                "description": "Alex Mercer and Jordan Hayes are both members of the Paranormal Military Squad and work closely together, sharing a mutual respect and understanding.",
                "out": "JORDAN HAYES",
                "strength": 9
            },
            {
                "description": "Al

In [54]:
#let's test the api with a generic prompt
client = Together()

stream = client.chat.completions.create(
  model="deepseek-ai/DeepSeek-R1",
  messages=[{"role": "user", "content": "What are the top 3 things to do in New York?"}],
  stream=True,
)

for chunk in stream:
    if len(chunk.choices)>0:
        print(chunk.choices[0].delta.content or "", end="", flush=True)

<think>
Okay, so I need to figure out the top three things to do in New York. Let's start by recalling what New York is famous for. There's the Statue of Liberty, Central Park, Times Square, Broadway shows, museums like the Metropolitan Museum of Art, maybe the Empire State Building? Also, places like the High Line or the 9/11 Memorial.

Wait, but which ones are the most popular? I think Statue of Liberty is a big one because it's an iconic symbol. Central Park is a huge park in the middle of Manhattan, so that's probably a must-see. Times Square is always buzzing with activity and lights, so that's likely on the list. But then again, there are museums. The Met is one of the largest art museums. The Empire State Building offers great views. The High Line is a newer attraction, an elevated park. Maybe Rockefeller Center? Or the Brooklyn Bridge?

I should check which attractions are most recommended by travel sites. But since I can't access the internet, I have to rely on my existing kno

In [55]:

#Let's create a helper function to pull out the <think> so we can do formatting and what not on the rest of the resposne
def parse_deepseek_response(input_string):
  """
  Parses a string containing <think> tags and extracts the content.

  Args:
    input_string: The string to parse.

  Returns:
    A dictionary with two keys:
      - "think": The content between the <think> tags.
      - "content": The rest of the string.
  """

  # Use regular expression to find the content within the <think> tags
  think_match = re.search(r"<think>(.*?)</think>", input_string, re.DOTALL)
  think_content = think_match.group(1).strip() if think_match else ""

  # Remove the <think> section from the original string
  content = re.sub(r"<think>.*?</think>\s*", "", input_string, flags=re.DOTALL).strip()

  return {"think": think_content, "content": content}

# Example usage
input_string = """
<think>

</think>

The color of the sky we see during the day is a result of the atmospheric absorption and scattering of sunlight. The short wavelength blue light, which is more easily absorbed by the atmosphere, passes through larger particles in the atmosphere, such as air molecules, before reaching us. At night, when the sun has set and the atmosphere is thinner, other wavelengths are scattered toward us by tiny particles like water droplets and small dust particles in the air. This scattering of light gives us a different color for the sky at night.
"""

result = parse_deepseek_response(input_string)
print(result)

{'think': '', 'content': 'The color of the sky we see during the day is a result of the atmospheric absorption and scattering of sunlight. The short wavelength blue light, which is more easily absorbed by the atmosphere, passes through larger particles in the atmosphere, such as air molecules, before reaching us. At night, when the sun has set and the atmosphere is thinner, other wavelengths are scattered toward us by tiny particles like water droplets and small dust particles in the air. This scattering of light gives us a different color for the sky at night.'}


In [60]:
#these functions will call together api based on our prompt templates and input


#input string of the model name and the prompt content
async def generate_content_with_attachment(the_model,the_messages):

        
    together = Together()
    response = together.chat.completions.create(
        messages=the_messages,
        model=the_model
    )

    content = response.choices[0].message.content
    parsed_response = parse_deepseek_response(content)
    return {"response":response, "think": parsed_response["think"],"content":parsed_response["content"]}
             



#this generates the prompt for a std RAG approach where we are supplying the document with the question
async def generate_std_LLM_messages(the_db,the_question):

    get_the_rag_context_surql = "fn::get_related_chunks('{0}')"
    
    rag_outcome = await the_db.query(get_the_rag_context_surql.format(the_question))


    messages = [
            {
                "role": "system",
                "content": DEEPSEEK_TOGETHERAI_STD_RAG_PROMPT.format(
                        documents = rag_outcome)
            },
             {
                "role": "user",
                "content": the_question
            }
            ]
    return messages


#this function will call the LLM with a std RAG approach prompt 
async def get_std_LLM_response(the_db,the_model,the_question):
    
    messages = await generate_std_LLM_messages(the_db,the_question)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
        
    response = await generate_content_with_attachment(the_model,messages)
    return response

   
#this generates the prompt for a graphRAG approach where we are supplying the document and knowledge graph with the question
#it calls the SurQL function get_related_entities to do a vector search based on the input question
async def generate_graph_RAG_messages(the_db,the_question):

    
    get_the_rag_context_surql = "fn::get_related_chunks('{0}')"
    
    rag_outcome = await the_db.query(get_the_rag_context_surql.format(the_question))

    
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    
    kg_outcome = await the_db.query(get_the_graph_context_surql.format(the_question))

    messages = [
            {
                "role": "system",
                "content": DEEPSEEK_TOGETHERAI_GRAPH_RAG_PROMPT.format(
                    knowledge_graph = kg_outcome,
                    documents = rag_outcome)
            },
             {
                "role": "user",
                "content": the_question
            }
            ]
    return messages
    
    
#this function will call the LLM with a prompt that includes the text of the original document
#plus the relevant entities and relationships from the knowledge graph based on a vector search
async def get_graph_RAG_response(the_db,the_model,the_question):
    
    messages = await generate_graph_RAG_messages(the_db,the_question)
    messages_to_log.append(messages)
    
    with open(chat_file, "w") as f:
        json.dump(messages_to_log, f)
    
    response = await generate_content_with_attachment(the_model,messages)
    return response
    
       

In [57]:

#specify the model to use

#model = "deepseek-r1:1.5b"
model = "deepseek-ai/DeepSeek-R1"


#retreive the input document to a string
with open(input_file) as f:
    document_text = f.read()




In [58]:
#Let's see how the prompts will look
#You can copy and paste these into any LLM chat engine to test

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    q = "Who is the main character and what motivates them?"
    
    print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")
    messages = await generate_std_LLM_messages(db,q)
    
    
    print(messages)
    
    print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

    messages = await generate_graph_RAG_messages(db,q)
    print(messages)
 
   






 ------ STD RAG -------------------------




[{'role': 'system', 'content': '\n-Goal-\nYou are a researcher who is providing answers to a question for your client based on documents provided you. Answer the questions to the best of your ability given the documents. The document text will be provided between the <documents> delimiters.\n\nThis is the data to respond to:\n<documents>\n[{\'result\': [{\'chunk\': \'conspiracies and furtive movements. But in the command center, a delicate tapestry of light and sound was being woven as the echoes of cosmic dialogue resonated through the high-tech enclave. Dr. Jordan Hayes, now leading the efforts, called out from their workstation, "I’ve isolated the signal\\\'s harmonics. It\\\'s more than a call; it\\\'s a song, an interstellar siren’s call." Alex Mercer, steady and resilient in the face of the incomprehensible, acknowledged with a quiet nod, "A song that we need to learn—quickly." His eyes, heavy with responsibility, scanned the room

In [ ]:
#Lets test a question...

q = "Who is the main character and what motivates them? Format your response in bullet points."

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    
    response = await get_std_LLM_response(db,model,q)


In [68]:

print ("\n ------ Was there an error or did it complete? -------------------------\n")
#  Length = "length"
#  StopSequence = "stop"
#  EOS = "eos"
#  ToolCalls = "tool_calls"
#  Error = "error"
#  Null = ""
#stop: This is the most common reason. It means the model generated text until it reached a natural stopping point, such as the end of a sentence or a logical conclusion. It could also indicate that the model reached a predefined maximum number of tokens.
#length: The model reached the maximum number of tokens specified in the request parameters.
#eos: The model encountered an end-of-sequence token, which signals the end of the generated text.
#error: An error occurred during the generation process. This could be due to various reasons, such as invalid input, model issues, or server problems.
#null: This usually means the generation is still in progress and hasn't finished yet.

print(response["response"].choices[0].finish_reason)

      
print ("\n ------ Think -------------------------\n")
#this is Deepseek's custom "thinking" 
print(response["think"])
print ("\n ------ Content -------------------------\n")
#For the rest of the content let's clean it up with a bit of Markdown
display(Markdown(response["content"]))


 ------ Was there an error or did it complete? -------------------------

FinishReason.StopSequence

 ------ Think -------------------------

Okay, let me start by trying to figure out who the main character is here. The user provided some documents, which are chunks of text from a story. They want to know the main character and their motivations, in bullet points.

Looking at the first document chunk, the names mentioned are Dr. Jordan Hayes, Alex Mercer, Sam Rivera, and Taylor Cruz. All of them seem to be part of a team working on decoding an alien signal. The story seems to be a group effort, but maybe one of them stands out as the main character.

In the first chunk, Dr. Jordan Hayes is mentioned as leading the efforts. Then in other chunks, Alex Mercer is interacting with Jordan, asking questions, and seems to be in a leadership role too. Taylor Cruz is giving commands and cautioning the team. Sam Rivera is working on technical aspects. But in the third chunk, Alex Mercer's eyes 

- **Main Character**: Alex Mercer  
- **Role**: A strategic leader within the team, focused on decoding and interacting with an extraterrestrial signal.  
- **Motivations**:  
  - **Mission Success**: Driven to ensure the team’s efforts to communicate with the alien intelligence succeed, despite unprecedented challenges.  
  - **Cautious Strategy**: Prioritizes careful interaction to avoid provoking conflict or unforeseen consequences.  
  - **Expanding Human Understanding**: Committed to navigating uncharted realities, balancing military discipline with the need for innovative dialogue.  
  - **Team Coordination**: Acts as a stabilizing force, ensuring collaboration between experts (e.g., Dr. Jordan Hayes’ technical work, Sam Rivera’s interface analysis, and Taylor Cruz’s protocol enforcement).  

*Note: While the narrative features an ensemble team, Alex Mercer is portrayed as the central figure guiding decisions and responses to the cosmic threat/opportunity.*

In [69]:
#and now let's see how the graph RAG will look...

async with AsyncSurrealDB(url) as db:
    await db.sign_in( u,  p) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q)
    

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


In [70]:


print(response["response"].choices[0].finish_reason)

      
print ("\n ------ Think -------------------------\n")
#this is Deepseek's custom "thinking" 
print(response["think"])
print ("\n ------ Content -------------------------\n")
#For the rest of the content let's clean it up with a bit of Markdown
display(Markdown(response["content"]))

FinishReason.StopSequence

 ------ Think -------------------------

Okay, let's tackle this question. The user wants to know who the main character is and what motivates them, using the provided documents and knowledge graph. 

First, I'll look at the main characters mentioned. The documents mention Alex Mercer, Dr. Jordan Hayes, Sam Rivera, and Taylor Cruz. All seem to be part of the Paranormal Military Squad involved in Operation: Dulce. But the question is about the main character. 

Looking at the knowledge graph, each character has their own entry. The strength of relationships might indicate importance. For example, Alex Mercer has a strength of 10 when related to himself, which suggests he's a central figure. Also, the descriptions mention that he's a skilled leader with a strong sense of responsibility.

In the documents, Alex is frequently mentioned as leading the team, interacting with others, and making strategic decisions. For example, in the documents, Alex is described as

- **Main Character**: **Alex Mercer**  
  - **Role**: Leader of the Paranormal Military Squad during Operation: Dulce, responsible for strategic decision-making and team coordination.  
  - **Motivations**:  
    - **Duty and Responsibility**: Driven by a profound sense of obligation to protect humanity and ensure mission success.  
    - **Unprecedented Discovery**: Committed to deciphering the alien signal, viewing it as a transformative opportunity for human understanding of extraterrestrial intelligence.  
    - **Team Stewardship**: Focused on maintaining cohesion and focus within his team amidst high-stakes, unpredictable challenges.  
    - **Strategic Resolve**: Motivated by the need to balance caution with progress, ensuring communication with the signal remains controlled and deliberate.  

---

**Key Supporting Evidence**:  
- The knowledge graph highlights Alex as a "skilled leader" with a "strong sense of responsibility" and the highest relational strength (10) in his role.  
- Documents emphasize his leadership in tense situations (e.g., "watching his team work tirelessly," "navigate by starlight").  
- Interactions with the team (e.g., urging caution, resolving to "echo [the signal's] patterns") underscore his motivation to bridge human and alien intelligence responsibly.

In [72]:

#these helper functions will run both approaches on the same question and print them for us

async def print_response(response):
    
    await print_reason(response["response"].choices[0].finish_reason)
    await print_think(response["think"])
    await print_content(response["content"])

async def print_reason(done_reason):
    if done_reason=="stop":
        return

    else:
        print(f"""!!There was an issue generating try again!!!!!

 -----------------  {done_reason} -----------------

        
    #stop: This is the most common reason. It means the model generated text until it reached a natural stopping point, such as the end of a sentence or a logical conclusion. It could also indicate that the model reached a predefined maximum number of tokens.
    #length: The model reached the maximum number of tokens specified in the request parameters.
    #eos: The model encountered an end-of-sequence token, which signals the end of the generated text.
    #error: An error occurred during the generation process. This could be due to various reasons, such as invalid input, model issues, or server problems.
    #null: This usually means the generation is still in progress and hasn't finished yet.
        """)
    
    print(response["response"].done_reason)

async def print_think(think):
    print(f"""<think>
{think}
</think>""")


async def print_content(content):
    print ("<content>")
    display(Markdown(content))
    print ("</content>")

    
async def process_and_display_RAG_comparison(q):
    async with AsyncSurrealDB(url) as db:
        await db.sign_in( u,  p) 
        await db.use(n, d)
        
        print ("""------ STD RAG -------------------------
""")
        std_response = await get_std_LLM_response(db,model,q)
        await print_response(std_response)
        
        print ("""------ GRAPH RAG -------------------------
""")
        graph_response = await get_graph_RAG_response(db,model,q)
        await print_response(graph_response)
       
    


In [73]:

#Let's test a question that asks about the main character's relationships
q = "What places and people does Alex Mercer hold dearest? Format your response in bullet points. Format your response in bullet points."
await process_and_display_RAG_comparison(q)



------ STD RAG -------------------------

<think>
Okay, I need to find out what places and people Alex Mercer holds dearest based on the provided documents. Let's start by going through each chunk of text carefully. 

First, in the first chunk, Alex Mercer is working with Dr. Jordan Hayes and others. They're in a dimly lit room with a decryption machine. The team includes Taylor Cruz and Sam Rivera. The interactions here show collaboration and respect. Alex asks Jordan about progress, indicating he values their expertise.

Next, in the second chunk, there's mention of the Paranormal Military Squad. Alex is part of this team, and they're communicating with unknown entities. The documents mention the team's importance, so maybe the squad itself is dear to Alex.

In the third chunk, the team is discussing coordinates and a warning. Alex is decisive here, focusing on understanding intent. His role as a leader in the team is emphasized, suggesting he values his team members.

The fourth chu

- **People**:  
  - **Dr. Jordan Hayes** (trusted colleague and collaborator, valued for their analytical skills and shared responsibility in high-stakes missions).  
  - **Taylor Cruz** (commanding ally, respected for their strategic oversight and focus on mission integrity).  
  - **Sam Rivera** (zealous astrotechnician, recognized for technical expertise and enthusiasm in confronting the unknown).  

- **Places**:  
  - **The command center/dimly lit decryption room** (primary workspace where critical missions unfold and first contact with alien intelligence is pursued).  
  - **Underground facilities** (implicitly tied to mission-critical operations, such as descending into secure locations for answers).  
  - **Paranormal Military Squad’s operational hubs** (symbolize duty and humanity’s role in cosmic dialogue).  

*Note: Personal relationships or non-mission-centric locations (e.g., family, home) are not mentioned in the provided documents.*

</content>
------ GRAPH RAG -------------------------

<think>
Okay, let's see. The user is asking for the places and people that Alex Mercer holds dearest. They want bullet points, so I need to make sure I structure the answer that way.

First, I'll check the knowledge graph provided. In the knowledge graph under Alex Mercer's entry, there are several RELATED_TO entries. Let me parse those. 

For people, the knowledge graph mentions Taylor Cruz, Jordan Hayes, and Sam Rivera. The relationships have strengths of 8, 9, and 7 respectively. The descriptions note that Jordan Hayes and Alex share mutual respect, and Sam is mentored by Alex. Taylor Cruz is a colleague but the strength is slightly lower. 

Regarding places, Dulce Base is mentioned with a strength of 9, and Washington D.C. with a strength of 6. Since the user is asking for places he holds dearest, Dulce Base seems more significant. Washington D.C. is where he reports to, but maybe not as dear as the operational base.

Looking a

- **People Alex Mercer Holds Dearest:**  
  - **Dr. Jordan Hayes**: A trusted colleague and fellow member of the Paranormal Military Squad. Their relationship is marked by mutual respect and collaborative problem-solving during high-stakes missions (strength: 9).  
  - **Sam Rivera**: A mentee and tech-savvy ally. Alex values Sam’s curiosity and adaptability, often relying on their expertise in critical operations (strength: 7).  
  - **Taylor Cruz**: A commanding officer and teammate. Though their dynamic is professional, Alex respects Taylor’s authority and leadership (strength: 8).  

- **Places Alex Mercer Holds Dearest:**  
  - **Dulce Base**: The clandestine facility where pivotal operations unfold. It serves as the epicenter of Alex’s groundbreaking work with his team (strength: 9).

socket.send() raised exception.
socket.send() raised exception.


</content>


In [74]:

# a bit of struture and creative calculation around his relationships
q = """
Create a table of people that Alex communicates with:

In the first column indicate the person. 
In the 2nd column list the frequency he has scenes with them.
In the 3rd column list the places they had scenes together ordered by importance of the place and scene.
In the 4th column indicate the strength of the bond between the people.
In the 5th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""
await process_and_display_RAG_comparison(q)

------ STD RAG -------------------------

<think>
Okay, let's tackle this query. The user wants a table of people Alex Mercer communicates with, based on the provided documents. First, I need to extract all the names mentioned in the document chunks.

Looking through the chunks, the names that come up are Sam Rivera, Taylor Cruz, Jordan Hayes. There's also mention of a Paranormal Military Squad team, but that's a group, not an individual. The chapters and scenes are in Dulce Base's command center. 

Next, I need to determine the frequency of scenes with each person. From the text, each team member is mentioned in several chunks. Sam is in two chunks, Jordan in three, Taylor in two. The frequency is estimated by the number of times they appear in scenes with Alex.

For the places they had scenes together, all interactions seem to be in Dulce Base's command center. The importance of the scene would be based on the activities described, like initiating protocols, decoding signals, etc. Th

```markdown
| Person         | Scene Frequency | Key Locations & Scenes                                                                 | Bond Strength (1-10) | Interaction Weight Score (1-10) |
|----------------|-----------------|---------------------------------------------------------------------------------------|-----------------------|----------------------------------|
| **Sam Rivera** | 3 scenes        | 1. Dulce Base command center (adjusting cosmic translators)<br>2. Mainframe room    | 7                     | 8                                |
| **Taylor Cruz**| 3 scenes        | 1. Command center (protocol initiation)<br>2. Strategic monitoring station             | 8                     | 8                                |
| **Jordan Hayes**| 4 scenes       | 1. Mainframe room (decoding alien code)<br>2. Command center (collaborative analysis | 9                     | 9                                |
```

### Notes:
1. **Scene Frequency**: Based on direct interactions described in the text (shared command center actions, collaborative troubleshooting, and strategic coordination).
2. **Key Locations**: 
   - *Dulce Base command center* is central to all interactions (most critical).
   - *Mainframe room* is secondary but pivotal for technical work.
3. **Bond Strength**: 
   - Strongest with **Jordan Hayes** (collaborative decoding, shared leadership in analysis).
   - Slightly weaker with **Sam Rivera** (technical support role).
4. **Interaction Weight**: 
   - Highest for **Jordan Hayes** due to recurring pivotal roles in narrative progression.
   - **Taylor Cruz** scores highly for strategic oversight but less direct technical involvement.

</content>
------ GRAPH RAG -------------------------

<think>
Okay, let me tackle this query step by step. The user wants a table of people that Alex Mercer communicates with, including their names, frequency of scenes, places they've been together ordered by importance, bond strength, and total interaction weight.

First, I need to look at the provided documents and the knowledge graph. The knowledge graph has entities like ALEX MERCER, JORDAN HAYES, SAM RIVERA, TAYLOR CRUZ, and TAYLOR CRUZ with their relationships. The documents mention interactions in Dulce Base and references to Washington as a command center.

Starting with the knowledge graph, Alex Mercer is related to Jordan Hayes, Sam Rivera, and Taylor Cruz. Each of these relationships has a strength score. For example, Jordan Hayes has a strength of 9, Sam Rivera 7, and Taylor Cruz 8. Wait, wait, looking back: the knowledge graph shows Alex's RELATED_TO links. For Jordan, it's strength 9; Sam, 7; Taylor, 8 (but in the knowle

```markdown
| Person         | Frequency | Places (Ordered by Importance) | Bond Strength | Total Weight |
|----------------|-----------|----------------------------------|---------------|--------------|
| Jordan Hayes   | 5         | Dulce Base                       | 9             | 45           |
| Taylor Cruz    | 3         | Dulce Base                       | 8             | 24           |
| Sam Rivera     | 3         | Dulce Base                       | 7             | 21           |
```

### Key Breakdown:
1. **Frequency**: Derived from direct co-occurrence in scenes across the document chunks.  
2. **Places**: All interactions occur at **Dulce Base**, the central hub of operations.  
3. **Bond Strength**: Based on the knowledge graph's explicit relationship strength scores.  
4. **Total Weight**: Calculated as `Frequency × Bond Strength` to quantify interaction significance.  

This table prioritizes Jordan Hayes as Alex's closest collaborator, followed by Taylor Cruz and Sam Rivera, reflecting their roles and narrative weight in the story.

</content>


In [75]:
# Let's see what that looks like from the places perspective
q = """
Create a table of places that Alex visits:

In the first column indicate the place name. 
In the 2nd column indicate the frequency he visits them.
In the 3rd column list the people who were there with him by strength of the bond of the people.
In the 4th column indicate the importance of the place.
In the 5th column describe the location in 255 characters or less.
In the 6th column score the total weight of the interactions across the scenes in aggregate.

output the response in a table format using markdown language.
"""
await process_and_display_RAG_comparison(q)

------ STD RAG -------------------------

<think>
Okay, let's tackle this query step by step. The user wants a table with specific information about the places Alex Mercer visits based on the provided documents.

First, I need to extract all mentions of places from the given documents. Scanning through the chunks, the main location mentioned is Dulce Base, specifically the command center and mainframe room. There's also a mention of a brewing storm outside, but that's part of the setting rather than a place Alex visits.

Next, the frequency of visits. Since the documents focus on activities within Dulce Base's command center and mainframe room, and there's no indication of other locations, I can infer that Alex is frequently there. The text describes multiple scenes in these areas, so "Multiple times" or "Consistently present" would be accurate.

For the people present, the team members mentioned are Sam Rivera, Jordan Hayes, Taylor Cruz, and others referred to as "team" or "agents". T

```markdown
| Place Name         | Visit Frequency | People Present (by bond strength)       | Importance (1-10) | Location Description (≤255 chars)                                                                 | Interaction Weight (1-10) |
|--------------------|------------------|------------------------------------------|--------------------|---------------------------------------------------------------------------------------------------|----------------------------|
| Dulce Base Command Center | Multiple times   | Sam Rivera, Jordan Hayes, Taylor Cruz, Paranormal Military Squad agents | 9.5                | A subterranean military facility with advanced tech for interstellar communication, bathed in blue light. Shielded, high-security. | 9.2                        |
``` 

**Notes:**  
- *Dulce Base Command Center* is the only explicitly named location in the documents. Alex is consistently present here.  
- **Bond strength** order reflects direct collaboration frequency (Sam/Jordan = technical roles, Taylor = command oversight).  
- **Importance** reflects its role as the central hub for alien communication and Earth's defense.  
- **Interaction weight** aggregates the intensity of dialogue, teamwork, and stakes across scenes (e.g., decoding alien signals, protocol activation).  
- No other locations are mentioned in the provided text chunks.

</content>
------ GRAPH RAG -------------------------

<think>
Okay, let's tackle this query. The user wants a table of places Alex Mercer visits, with several specific columns. First, I need to look at the provided documents and knowledge graph.

Looking at the knowledge graph, Alex Mercer is related to Dulce Base and Washington D.C. The documents mention Dulce Base several times, especially in the context of the command center where the team works. Washington D.C. is mentioned in the 'RELATED_TO' relationships as a place where Alex and others report to.

Now, let's check the places. The knowledge graph shows Alex is related to Dulce Base with strength 9 and Washington with strength 6. Dulce Base is clearly the main location in the documents, described as an underground facility. Washington is mentioned as a reporting location but not as a place he's physically present in the scenes provided.

The people with Alex at Dulce Base are Jordan Hayes (strength 9), Sam Rivera (7), and Taylor

```markdown
| Place Name    | Frequency                  | People Present (Strength)     | Importance               | Description                                                                 | Interaction Weight |
|---------------|---------------------------|--------------------------------|--------------------------|-----------------------------------------------------------------------------|--------------------|
| Dulce Base    | Frequent (primary base)   | Jordan Hayes (9), Sam Rivera (7), Taylor Cruz (8) | Critical (operational hub) | A subterranean military facility with advanced tech for interstellar communication, shielded from public view. | 24                 |
```

### Notes:
1. **Dulce Base**:  
   - **Frequency**: Central to all operations in the provided scenes.  
   - **People**: Team members present with Alex are listed by relationship strength (from the knowledge graph).  
   - **Importance**: Critical for interstellar communication and classified missions.  
   - **Interaction Weight**: Sum of relationship strengths (Jordan: 9, Sam: 7, Taylor: 8).  

Washington D.C. is mentioned in the knowledge graph as a command node, but no explicit visits by Alex are documented in the provided text. Only Dulce Base is directly tied to his on-scene presence.

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.s

</content>


In [76]:
#Let's try a creative endeavor
q = """
    Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Describe this cover in 20 words or less.
"""
await process_and_display_RAG_comparison(q)


------ STD RAG -------------------------

<think>
Alright, the user wants a book cover scene based on Alex Mercer's strongest relationships and key plot locations, emphasizing the most emotionally charged moment. Let me parse the documents provided.

The documents mention the team at Dulce Base's command center, interacting with an alien signal. Key characters are Alex Mercer, Jordan Hayes, Taylor Cruz, and Sam Rivera. The scenes describe their tense communication efforts, the weight of their mission, and the significance of their roles as liaisons to an extraterrestrial entity.

The emotionally charged moments seem to be when they first connect with the alien signal, the team's realization of their role as Earth's interpreters, and the tension between breakthrough and disaster. The command center is a central location, described with monitors showing alien scripts, a theater-like atmosphere, and the team's unity.

Alex's strongest relationships are with Jordan (trust, companionship) a

**Cover Description**:  
Intense team huddles around glowing alien script monitors in Dulce Base, cosmic shadows looming as Earth's fate trembles in their silent resolve.

</content>
------ GRAPH RAG -------------------------

<think>
Okay, let's tackle this query. The user wants a book cover description based on Alex Mercer's strongest relationships and the most important places in the plot. The answer has to be 20 words or less.

First, I need to look at Alex Mercer's strongest relationships from the knowledge graph. His top connections are with Jordan Hayes (strength 9), Taylor Cruz (strength 8), Sam Rivera (strength 7), and Dulce Base (strength 9). Places important to the plot are Dulce Base and Washington, but Dulce Base has a higher strength in his relationships.

The most emotionally charged scene would likely involve these key people and the primary location. From the documents, there's a tense scene in the Dulce Base command center where the team is interacting with an alien signal. The emotional charge comes from their roles as protectors and interpreters, the high stakes, and their unity.

Mentions of "theatre for performing the elaborate chor

**Book Cover Description**:  
Alex Mercer and team huddled in Dulce Base command center, alien script glowing on monitors, shadows of unity and cosmic uncertainty.

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


</content>


In [77]:


q = """Based on Alex Mercer strongest relationships and the places most important to the plot choose a
    scene that is the most emotionally charged and important to make the cover of the book. 
   Draw this cover in the style of an 8-bit video game.
"""

await process_and_display_RAG_comparison(q)

------ STD RAG -------------------------

<think>
Okay, I need to figure out which scene from the provided documents is the most emotionally charged and important for the book cover, focusing on Alex Mercer's strongest relationships and key plot locations.

First, I'll review the documents to identify the main characters and settings. The main characters mentioned are Dr. Jordan Hayes, Alex Mercer, Sam Rivera, and Taylor Cruz. They're working in a high-tech command center, likely part of the Dulce Base. The plot revolves around decoding an alien signal, which is described as a song or a cosmic dialogue. The team is under pressure, dealing with the implications of making contact with an alien civilization.

Alex Mercer is depicted as a steady leader with a sense of responsibility. His strongest relationships seem to be with Jordan Hayes, given their collaboration in analyzing the signal, and the team as a whole. There's a mention of pivotal moments in the command center where the team w

**Cover Design Concept for 8-Bit Book Cover:**

**Scene:**  
The pivotal moment from **Chapter 12** in Dulce Base’s command center, where Alex Mercer, Jordan Hayes, Sam Rivera, and Taylor Cruz huddle around flickering monitors displaying alien script. This scene captures the team’s awe, tension, and unity as they decode the cosmic signal—a turning point where humanity’s first interstellar dialogue hangs in the balance.

---

**Visual Breakdown (8-Bit Style):**  
1. **Setting**:  
   - Retro pixel-art rendition of Dulce Base’s underground command center.  
   - Steel walls with glowing blue and green terminals, radar screens, and scrolling alien symbols (blocky, symbolic code).  
   - A massive central monitor dominates the background, radiating golden starbursts of the “cosmic signal.”  

2. **Characters (Pixel Art)**:  
   - **Alex Mercer**: Central figure, standing tall with a stoic, determined expression. Rendered in dark military garb (black/gray pixels) with a badge bearing the Paranormal Military Squad insignia.  
   - **Jordan Hayes**: To Alex’s left, leaning over a console. Lab coat (white pixels), glasses reflecting screen light, fingers hovering over a keyboard.  
   - **Sam Rivera**: To Alex’s right, wide-eyed and gesturing excitedly toward the main monitor. Wearing a tech vest (green pixels), hands animated.  
   - **Taylor Cruz**: Behind Alex, arms crossed, face stern but watchful. Uniform (navy pixels) with a commander’s insignia.  

3. **Emotional Tone**:  
   - **Color Palette**: Neon blues and cyans for tech elements, contrasting with warm golds from the cosmic signal. Dark grays for the base’s structure to emphasize secrecy.  
   - **Lighting**: Monitors cast a dramatic glow, illuminating the team’s faces. The alien script pulses rhythmically, symbolizing the “song” they’re decoding.  

4. **Foreground/Details**:  
   - 8-bit stars and constellations shimmer in the background through a faux “window” (hinting at space beyond the base).  
   - A retro “dialogue box” at the bottom reads: *“WE’RE CONVERSING WITH THE STARS”* in chunky pixel font.  
   - Subtle “glitch” effects on the edges of the screen, blending sci-fi and retro gaming aesthetics.  

---

**Why This Scene?**  
- **Alex’s Relationships**: Highlights his role as leader alongside Jordan (trusted ally), Sam (optimistic counterpart), and Taylor (cautious advisor).  
- **Plot Significance**: Dulce Base is the story’s nerve center, and this moment bridges human curiosity and cosmic mystery.  
- **Emotional Charge**: The team’s shared hope, fear, and wonder crystallize here—perfect for a cover that invites intrigue.  

**Style Nods**:  
Think *Stranger Things* meets *Undertale*, blending 80s sci-fi mystery with modern pixel-art warmth.

</content>
------ GRAPH RAG -------------------------

<think>
Okay, let's tackle this query. The user wants a book cover designed based on Alex Mercer's strongest relationships and the key places in the plot, in an 8-bit video game style. First, I need to figure out who Alex is closely connected to. The knowledge graph shows he has strong links to Jordan Hayes (strength 9), Taylor Cruz (8), Sam Rivera (7), and Dulce Base (9). The Dulce Base is the main location.

The documents mention intense scenes in the command center of Dulce Base with the team analyzing alien signals. Jordan is decoding the signal's harmonics, Alex is leading, Sam finds patterns, and Taylor cautions them. This scene seems emotionally charged as they're on the brink of contact, which is a pivotal moment.

For the 8-bit style, I need pixel art elements. The command center setup with monitors showing alien code, team members at consoles. Highlight Alex's leadership by placing him centrally. Jordan and Sam working in

**Book Cover Concept:**  
**Title:** *Operation Dulce: Echoes of the Cosmic Choir*  
**8-Bit Art Style:** Retro pixel art with a dark sci-fi palette (neon greens, deep blues, and stark blacks)  

---

### **Scene Highlighted:**  
**Location:** *Dulce Base Command Center* (Alex Mercer’s strongest location tie, **strength 9**)  
**Key Characters:**  
- **Alex Mercer (center)** – Pixel-art depiction with a determined stance, eyes glowing faintly (symbolizing leadership and cosmic connection).  
- **Jordan Hayes (left)** – Hunched over a flickering terminal, hands radiating light as alien code scrolls across screens.  
- **Sam Rivera (right)** – Manipulating a holographic interface of fractal-like alien symbols.  
- **Taylor Cruz (background)** – Silhouetted in a doorway, tense posture, clutching a data pad with a red "SECURE" warning.  

---

### **Emotional Core:**  
The team’s **first successful decryption of the alien signal** (the "interstellar siren’s call") as described in the documents. The scene pulses with tension:  
- Alien glyphs spiral dynamically in the background, forming a glowing portal-like shape (symbolizing cosmic communication).  
- Pixel-art monitors show waveforms and the *“repeating motif”* of the signal.  
- A subtle 8-bit "countdown timer" at the top hints at impending danger (aligning with Taylor Cruz’s caution).  

---

### **Symbolism & 8-Bit Details:**  
- **Alex’s Leadership:** His pixelated figure is slightly larger, with a faint aura echoing the alien glyphs (representing his role as humanity’s bridge to the unknown).  
- **Dulce Base Aesthetic:** Exposed pipes, retro-futuristic CRT monitors, and a cracked ceiling revealing a starry sky (symbolizing secrets breaking into the open).  
- **Hidden Detail:** A tiny UFO sprite near the portal glyphs foreshadows the cosmic threat.  

**Color Palette:** Neon green (alien tech), electric blue (human tech), and blood-red accents (urgency).  

---

This cover encapsulates Alex’s pivotal relationships (team unity vs. Taylor’s wariness) and Dulce Base’s role as the crucible of humanity’s first interstellar dialogue. The 8-bit style evokes nostalgia while mirroring the story’s tension between analog humanity and unfathomable cosmic code.

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


</content>
